# Segmental retention models for representing the hydraulic properties of evolving structured soils

This notebook recreates Figs. 1 and 2.


In [ ]:
import segmental_models as svg
import vangenuchten as vg

import numpy as np
from scipy.constants import micro, milli, nano
import matplotlib.pyplot as plt

In [ ]:
# Constant A such that psi [length] = - A / r, where r is pore radius
A = 1.49e-5  # m^2 (square meters), see https://www.wolframalpha.com/input?i=2+%28water+surface+tension%29+%2F+%28earth+gravity%29+%2F+%28water+density%29

threshold_h = A / np.array([50 * nano, 1 * micro, 20 * micro, 0.5 * milli])

Global settings for plots

In [ ]:
plt.rcParams['figure.figsize'] = (4.5, 4.5 / 1.4)

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

h = np.logspace(-2.5, 3.5)

## Smooth step

In [ ]:
x = np.linspace(0, 1)

plt.plot(x, svg.smoothceiling(x), label='smooth ceiling', color=colors[4])
plt.plot(x, svg.smoothfloor(x), label='smooth floor', color=colors[0])
# plt.plot(x, svg.smoothstep(x), '--', label= 'smooth step', color= colors[1])
plt.xlabel('$x$')
plt.ylabel('$s(x)$')
plt.legend()

plt.tight_layout()

## Clipping functions

In [ ]:
plt.plot(h, h, 'k')
for row in svg.clipping(h, threshold_h):
    plt.plot(h, row, lw= 1.6, dashes= (10,1))

plt.xscale('log')
plt.yscale('log')
plt.xlabel('pressure head $h$ [m]')
plt.ylabel('pressure head $h$ [m]')

ymin, ymax = plt.ylim()
plt.vlines(threshold_h, ymin=ymin, ymax=ymax, colors='.8', linewidth= .5, zorder=-1)
plt.ylim((ymin, ymax))

plt.gca().invert_xaxis()
plt.gca().invert_yaxis()

plt.tight_layout()

# Add minus signs to tick labels
for ticks in (plt.xticks, plt.yticks):
    ticklabels = ticks()[1][2:-2]
    for Text in ticklabels:
        Text.set_text(Text.get_text().replace('10','\N{MINUS SIGN}10'))
    ticks(ticks()[0][2:-2], ticklabels)

plt.savefig('clipping_functions.pdf', bbox_inches="tight", pad_inches=.01)

## Retention components

In [ ]:
soil = {'n': 1.41, 'a': 2.0, 'mcr': 0.067, 'mcs': 0.45}

VG_mc = vg.mc(h, **soil)

sVG_comp = svg.comps(h, threshold_h, **soil)
sVG_mca = svg.mc(sVG_comp, threshold_h, **soil)[0]

In [ ]:
plt.plot(h, VG_mc, 'k', label='original')

for comp in sVG_mca:
    plt.plot(h, comp, lw= 1.6, dashes= (10,1))

plt.gca().add_line(
    matplotlib.lines.Line2D(
        [],
        [],
        color= '.65',
        dashes= (6,1),
        lw= 1.6,
        label='pore-size class\nsegments',
    )
)

plt.plot(h, soil['mcr'] + sVG_mca.sum(axis=0), label='sum of segments', color=colors[6])

_, _, ymin, ymax = plt.axis()
plt.vlines(threshold_h, ymin=ymin, ymax=ymax, colors='.8', linewidth= .5, zorder=-1)

plt.xscale('log')
plt.xlabel('pressure head $h$ [m]')
plt.ylabel(r'water content $\theta$ [-]')
plt.ylim((ymin, ymax))
plt.legend()
plt.gca().invert_xaxis()

plt.tight_layout()

# Add minus signs to tick labels
ticklabels = plt.xticks()[1][2:-2]
for Text in ticklabels:
    Text.set_text(Text.get_text().replace('10','\N{MINUS SIGN}10'))
plt.xticks(plt.xticks()[0][2:-2], ticklabels);

plt.savefig('Silt_Loam_retention_proper.pdf', bbox_inches="tight", pad_inches=.01)